This tutorial walks you through how to use the gizmo_analysis package to analyze and plot particle data from Gizmo simulation snapshot files and additional post-processing files. This assumes that you already have worked through gizmo_tutorial_read.ipynb, the tutorial on reading this particle data.

@author:
    Andrew Wetzel <arwetzel@gmail.com>
    Andrew Emerick <aemerick11@gmail.com>
    Isaiah Santistevan <ibsantistevan@ucdavis.edu>

In [ ]:
import gizmo_analysis as gizmo  # rename these packages for brevity
import utilities as ut  # rename these packages for brevity

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt


# Read the particle data

In [ ]:
# You can copy this jupyter notebook tutorial into a simulation directory (for example, m12i_r7100/)
# and run from there, or you can set simulation_directory below to point to any simulation directory
# and then run this notebook from anywhere.

# Use this is you are running from within a simulation directory
#simulation_directory = '.'

# Use this to point to a specific simulation directory, if you run this notebook from somwhere else
simulation_directory = '/Users/awetzel/work/research/simulation/gizmo/simulations/m12/m12i/m12i_r7100'


In [ ]:
# Read all particle species at z = 0

part = gizmo.io.Read.read_snapshots('all', 'redshift', 0, simulation_directory)

# Alternatley, read star and dark-matter particles at z = 0
# part = gizmo.io.Read.read_snapshots(['star', 'dark'], 'redshift', 0, simulation_directory)


# Computing the spatial profile of a property

A common task is to compute a radial profile of a given quantity, such as mass density, average age, median metallicity, etc.

The following functions in *utilities.particle* make this easier to do.

First, initiate an instance of SpeciesProfileClass.

As you initialize, choose your distance/radius binning scheme:
  distance limits, bin width, whether to use log scaling, number of spatial dimensions of profile

Refer to ut.binning.DistanceBinClass() for more.


In [ ]:
# linear binning from 0 to 20 kpc with 1 kpc bin width, assuming a 3-D profile

SpeciesProfile = ut.particle.SpeciesProfileClass(
    limits=[0, 20], width=1, log_scale=False, dimension_number=3)


In [ ]:
# Using this binning scheme, compute sum/histogram/density of mass of star particles in each bin
# This returns a bunch of summed properties via a dictionary

pro = SpeciesProfile.get_sum_profiles(part, 'star', 'mass')


In [ ]:
# You can supply a list of multiple species, and it will compute profiles for each
# thus, get_sum_profiles() returns a dictionary for each species
# 'baryon' is the sum total of stars + gas, 'total' is the sum total of all particle species

pro.keys()


In [ ]:
# The quantities that it stores in each bin

pro['star'].keys()


Alternately, you may want to compute profiles along a disk's cylindrical R or Z axes. If so, first define the dimensionality of the profile when you initiate the class.

In [ ]:
# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 2-D profile (along R)
SpeciesProfile = ut.particle.SpeciesProfileClass(
    limits=[0.1, 10], width=0.1, log_scale=True, dimension_number=2)


In [ ]:
# Set rotation=True to force the code to compute profiles along the principal axes (assuming that you assign them during read in)
# Use other_axis_distance_limits to limit the extent along the other axis,
# in this case, limit the Z axis to within +/- 1 kpc (in the profile, all distances are absolute)

pro = SpeciesProfile.get_sum_profiles(
    part, 'star', 'mass', rotation=True, other_axis_distance_limits=[0, 1])


In [ ]:
# Similarly, do this to compute profiles along disk's Z axis

# log binning from 0.1 to 10 kpc with 0.1 dex bin width, assuming a 1-D profile (along Z)
SpeciesProfile = ut.particle.SpeciesProfileClass(
    limits=[0.1, 10], width=0.1, log_scale=True, dimension_number=1)

# limit the R axex to [5, 8] kpc
pro = SpeciesProfile.get_sum_profiles(
    part, 'star', 'mass', rotation=True, other_axis_distance_limits=[5, 8])


In [ ]:
# get_statistics_profiles() computes various statistics of a property of particles in each bin below,
# it weights the property by the mass of each star particle (generally what you should do)
# this returns a bunch of useful statistics via a dictionary

pro = SpeciesProfile.get_statistics_profiles(
    part, 'star', 'age', weight_property='mass', rotation=True, other_axis_distance_limits=[5, 8])


In [ ]:
# the quantities that it stores in each bin

pro['star'].keys()


# Make an image

Here is a simple version of code for making an image. By default, it will align the axes with the stellar disk of the galaxy.


In [ ]:
def plot_image(
    part,
    species_name='star',
    weight_name='mass',
    part_indices=None,
    dimensions_plot=[0, 1],
    distance_max=20,
    distance_bin_width=0.1,
    rotation=True,
    host_index=0,
    image_limits=[None, None],
):
    '''
    Plot 2-D projected image of the positions of given partcle species.

    Parameters
    ----------
    part : dict
        catalog of particles at snapshot
    species_name : str
        name of particle species to plot
    weight_name : str
        property to weight positions by
    part_indices : array
        input selection indices for particles
    dimensions_plot : list
        spatial dimensions to plot
    distance_max : float or array
        maximum distance from center to plot [kpc]
    distance_bin_width : float
        size of pixel [kpc]
    rotation : bool or array
        whether to rotate particles
        if True, will rotate to align with principal axes defined by input species
    host_index : int
        index of host halo to get position and rotation of (if not input them)
    image_limits : list
        min and max limits to impose on image dynamic range (exposure)
    '''
    from matplotlib import colors

    dimen_label = {0: 'x', 1: 'y', 2: 'z'}

    # get distance limits for plot
    position_limits = [[-distance_max, distance_max] for _ in range(2)]
    position_limits = np.array(position_limits)

    # get array of particle indices (if not input)
    if part_indices is None or len(part_indices) == 0:
        part_indices = ut.array.get_arange(part[species_name]['position'].shape[0])

    # get positions relative to host galaxy center
    host_name = ut.catalog.get_host_name(host_index)
    if rotation is True:
        # rotate according to principal axes
        positions = part[species_name].prop(f'{host_name}.distance.principal', part_indices)
    else:
        # positions in (arbitrary) Cartesian x,y,z of simulation
        positions = part[species_name].prop(f'{host_name}.distance', part_indices)

    # weight particles by some property?
    weights = None
    if weight_name:
        weights = part[species_name].prop(weight_name, part_indices)

    # keep only particles within distance limits along each dimension
    masks = positions[:, dimensions_plot[0]] <= distance_max
    for dimen_i in dimensions_plot:
        masks *= (positions[:, dimen_i] >= -distance_max) * (positions[:, dimen_i] <= distance_max)

    # keep only positions and weights within distance limits
    positions = positions[masks]
    if weights is not None:
        weights = weights[masks]

    # get number of bins (pixels) along each dimension
    position_bin_number = int(np.round(2 * np.max(distance_max) / distance_bin_width))

    # get 2-D histogram
    hist_valuess, hist_xs, hist_ys = np.histogram2d(
        positions[:, dimensions_plot[0]],
        positions[:, dimensions_plot[1]],
        position_bin_number,
        position_limits,
        weights=weights,
    )
    # convert to surface density
    hist_valuess /= np.diff(hist_xs)[0] * np.diff(hist_ys)[0]

    # set color map
    if 'dark' in species_name:
        color_map = plt.cm.afmhot
    elif species_name == 'gas':
        color_map = plt.cm.afmhot
    elif species_name == 'star':
        color_map = plt.cm.afmhot

    fig = plt.figure()
    subplot = fig.add_subplot(111, facecolor='black')

    subplot.set_xlim(position_limits[0])
    subplot.set_ylim(position_limits[1])

    subplot.set_xlabel(f'{dimen_label[dimensions_plot[0]]} $\\left[ {{\\rm kpc}} \\right]$')
    subplot.set_ylabel(f'{dimen_label[dimensions_plot[1]]} $\\left[ {{\\rm kpc}} \\right]$')

    # make 2-D histogram image
    Image = subplot.imshow(
        hist_valuess.transpose(),
        norm=colors.LogNorm(),
        cmap=color_map,
        aspect='auto',
        interpolation='bilinear',
        extent=np.concatenate(position_limits),
        vmin=image_limits[0],
        vmax=image_limits[1],
    )

    fig.colorbar(Image)

    plt.show(block=False)


In [ ]:
plot_image(part, 'star', dimensions_plot=[0, 1])
plot_image(part, 'star', dimensions_plot=[1, 2])
plot_image(part, 'star', dimensions_plot=[0, 2])


# Exploring particle positions

In [ ]:
# Get cylindrical coordinates of star particles

positions = part['star'].prop('host.distance.principal.cylindrical')


In [ ]:
# Get 3-D cylindrical coordinates of particle 0: R [kpc physical], phi [radians], Z [kpc physical]

positions[0]


In [ ]:
# R coordinates [kpc physical] for all star particles

positions[:, 0]


In [ ]:
# Z coordinate [kpc physical] for particle 1234

positions[1234, 2]


In [ ]:
positions = part['star'].prop('host.distance.principal.cylindrical')
velocities = part['star'].prop('host.velocity.principal.cylindrical')
print(positions[:,0].size)


In [ ]:
masks_1 = (positions[:,0] < 20)
print(np.sum(masks_1))


In [ ]:
masks_2 = masks_1 * (positions[:,0] > 10)
print(np.sum(masks_2))


In [ ]:
masks_3 = masks_2 * (velocities[:, 0] < 0)
print(np.sum(masks_3))


In [ ]:
positions[masks_3]


In [ ]:
# Boolean (True/False) masks are a useful way to get particles of interest in some region
# For example, this will select particles withing cylindrical R = [4, 12] kpc and Z = [-1, 1] kpc

positions = part['star'].prop('host.distance.principal.cylindrical')

masks = (positions[:, 0] > 4) * (positions[:, 0] < 12)  # select along R
masks *= (positions[:, 2] > -1) * (positions[:, 2] < 1)  # select along Z

# print R and Z values
print(positions[masks, 0])
print(positions[masks, 2])


# Plot the distribution (histogram) of a property

Another common form of analysis is to examine the distribution (histogram) of some property, like gas density, temperature, or stellar age, or dark-matter velocity.

In [ ]:
# Here is a sample code that plots the distribution of some property.

def plot_property_distribution_simple(
    property_values,
    property_limits=[],
    property_bin_number=100,
    property_log_scale=True,
    axis_y_limits=[],
    axis_y_log_scale=True,
):
    '''
    Plot probability distribution of property.

    Parameters
    ----------
    property_values : array
        1-D array of values
    property_limits : list
        min and max limits of property
    property_bin_number : int
        number of bins within limits
    property_log_scale : bool
        whether to use logarithmic scaling for property bins
    axis_y_limits : list
        min and max limits for y axis
    axis_y_log_scale : bool
        whether to use logarithmic scaling for y axis
    '''
    if property_limits is None or not len(property_limits) > 0:
        property_limits = [np.min(property_values), np.max(property_values)]

    property_values_use = np.array(property_values)
    property_limits_use = np.array(property_limits)

    if property_log_scale:
        property_values_use = np.log10(property_values_use)
        property_limits_use = np.log10(property_limits)

    # set this false to return a histogram
    # set this true to return probability distribution
    density = False

    hist, bin_edges = np.histogram(
        property_values_use, property_bin_number, property_limits_use, density=density
    )

    if property_log_scale:
        bin_edges = 10**bin_edges

    bin_edges = bin_edges[:-1]

    # plot ----------
    fig = plt.figure()
    subplot = fig.add_subplot(111)

    if property_log_scale:
        subplot.set_xscale('log')
    else:
        subplot.set_xscale('linear')

    if axis_y_log_scale:
        subplot.set_yscale('log')
    else:
        subplot.set_yscale('linear')

    subplot.set_xlim(property_limits)
    if axis_y_limits is not None and len(axis_y_limits) > 0:
        subplot.set_ylim(axis_y_limits)

    subplot.set_xlabel('x label')
    subplot.set_ylabel('y label')

    subplot.plot(
        bin_edges,
        hist,
        alpha=0.8,
    )

    plt.show(block=False)


In [ ]:
# For example, plot the distribution of stellar ages

masks = part['star'].prop('host.distance.total') < 20  # [kpc] get stars in/near host galaxy

ages = part['star'].prop('age')[masks]

plot_property_distribution_simple(
    ages,
    [0, 13],
    property_bin_number=100,
    property_log_scale=False,
    axis_y_limits=[],
    axis_y_log_scale=True,
)


In [ ]:
# For another example, plot the distribution of stellar total velocities

masks = part['star'].prop('host.distance.total') < 20  # [kpc] get stars in/near host galaxy

velocities = part['star'].prop('host.velocity.total')[masks]

plot_property_distribution_simple(
    velocities,
    [0, 1000],
    property_bin_number=100,
    property_log_scale=False,
    axis_y_limits=[],
    axis_y_log_scale=True,
)


## Scatter plot of property A versus property B

Generally, the most useful/informative plots are to examine particles in bins of some x-quantity (like radial distance) and examine the mean and spread of some other y-property (like radial velocity) within that x-bin.

In [ ]:
# Here is an example: stellar radial velocity v distance

distances = part['star'].prop('host.distance.total')  # total (scalar) distance
velocities = part['star'].prop('host.velocity.principal.spherical')[:, 0]  # radial velocity

# define distance bins
distance_min = 0
distance_max = 300
distane_bin_width = 5
distance_bins = np.arange(distance_min, distance_max, distane_bin_width)

# arrays to store mean velocity and its standard deviation in each distance bin
vel_rad_ave = np.zeros(distance_bins.size)
vel_rad_std = np.zeros(distance_bins.size)

# loop through distance bins to get masks of particles in that bin
for distance_index, distance_bin in enumerate(distance_bins):
    masks = (positions > distance_bin) * (positions < distance_bin + distane_bin_width)
    # compile mean and standard deviation
    vel_rad_ave[distance_index] = np.mean(velocities[masks])
    vel_rad_std[distance_index] = np.std(velocities[masks])

# print results
#for distance_index, distance_bin in enumerate(distance_bins):
#    print('{} {:.2f} {:.2f}'.format(distance_bin, vel_rad_ave[distance_index], vel_rad_std[distance_index]))


In [ ]:
# Plot the above results for radial velocity versus distance,
# showing each particle as a point, and including a line that shows the mean trend versus distance,
# and a shaded region showing the standard deviation.

fig, ax = plt.subplots()
fig.set_size_inches(6, 4)

ax.set_ylim(-400, 400)  # set y-axis limits in [km/s]
ax.set_xlim(distance_min, distance_max)  # set x-axis limits in [kpc]
ax.set_xlabel('distance from galaxy [kpc]')  # labels
ax.set_ylabel('radial velocity [km/s]')

# plot radial position versus radial velocity for each particle
# because there are a lot of particles, you might want to sub-sample the arrays
# for an array arr, you can sub-sample every Nth element via arr[::N]
# this example selects every 100th particle within the distance limit
masks = (positions < distance_max)
sample_num = 100
distances_rad = positions[masks][::sample_num]
velocities_rad = velocities[masks][::sample_num]
# alpha sets the transparency, s sets the point size
ax.scatter(distances_rad, velocities_rad, alpha=0.2, s=1)

# lw sets the line width
ax.plot(distance_bins, vel_rad_ave, lw=3, color='black', label='mean')
ax.fill_between(distance_bins, vel_rad_ave + 0.5 * vel_rad_std, vel_rad_ave - 0.5 * vel_rad_std,
                alpha=0.5, color='black', label='std dev')

ax.legend(loc='best')


## 2-D histogram plot of property A versus property B

Instead of plotting a scatter plot (one point per particle), which can get unwieldy with this large number of particles, we instead can make a 2-D histogram, by binning particles into 2-D bins along both the x- and y-axes, then plotting a histogram (colormap) of the results.

In [ ]:
# Here is an example, 'hase diagram' for gas, of temperature versus density

from matplotlib import colors

masks = part['gas'].prop('host.distance.total') < 20  # [kpc] get gas in/near host galaxy

masks *= part['gas'].prop('number.density') > 1e-4  # get limited range of gas density
masks *= part['gas'].prop('number.density') < 1e4

# Make the plot
fig = plt.figure()
ax = plt.gca()

density_bins = np.logspace(-4, 12, 256)
temperature_bins = np.logspace(1, 7, 256)

ax.hist2d(
    part['gas'].prop('number.density')[masks],
    part['gas']['temperature'][masks],
    bins=[density_bins, temperature_bins],
    norm=colors.LogNorm(),
)

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(1e-4, 1e4)

ax.set_xlabel(r'number density [atoms cm$^{-3}$]')
ax.set_ylabel(r'temperature [K]')


# For more information

See gizmo_plot.py (which you can access here via gizmo.plot) for examples of analyzing and plotting particle data.

See utilities/particle.py (which you can acces here via ut.particle) for mid-level analysis functions that may be useful.

See other modules within utilities for low-level functions that may be useful.